In [31]:
import apache_beam as beam
import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
from apache_beam.pipeline import Pipeline
from apache_beam.io import ReadFromBigQuery
from apache_beam.io.gcp.internal.clients import bigquery
from typing import List, Tuple, Any

In [ ]:
class PivotRow(beam.DoFn):
    def process(self, element, k_fields, p_fields, v_fields):
        
        # start with the key_fields, unchanged
        pivoted_record = {k:v for k,v in element.items() if k in k_fields}
        
        # then update with new pivoted fields 
        for p_field in p_fields:
            for v_field in v_fields:
                col_name = element[p_field] + '_' + v_field
                col_name = col_name.replace(' ', '_')
                value = element[v_field]
                pivoted_record.update({col_name:value})

        yield pivoted_record

    def expand(self, records):
        
        schema_fields = self.schema_str.split(',')
        fields = [s.split(':')[0] for s in schema_fields if s.split(':')[0] not in self.k_fields]
        
        # result = {}

        for k_field in self.k_fields:
            for field in fields:
                # field_max = records | beam.Map(MaxFinder)
                field_max = (
                    records 
                    | field >> beam.GroupBy(lambda x: x[k_field])
                    .aggregate_field(lambda x: x[field], max, field)
                    )
            
                yield field_max

In [137]:
# class trydo():
#     def process(self, element):
# def mymax():


class try1(beam.PTransform):
    def expand(self, records):

        A = []

        for k_field in ['a']:

            for field in ['b','c','d']:

                fmax = (
                    records 
                    | 'filter'+field >> beam.Filter(lambda x: x[field] != None)
                    | "max"+field >> beam.GroupBy(lambda x: x[k_field]).aggregate_field(lambda y: y[field], max, field)
                    | "y"+field >> beam.Map(lambda r: {k_field: r[0], field: r[1]})
                    # | "tolist"+field >> beam.transforms.combiners.ToDict()
                    # | "asiter"+field >> beam.pvalue.AsIter()
                    )

                # A.append(fmax)

                return fmax#, fmax # error
            # return [fmax] #error
            # return fmax #ok

        # return beam.Create([A])


p = Pipeline(InteractiveRunner())

pcoll = p | "make" >> beam.Create([{'a':4,'b':1,'c':3,'d':6}, {'a':4,'b':None,'c':4,'d':3}, {'a':1,'b':6,'c':8,'d':None}])            

pc = pcoll | try1() | beam.Map(print)


In [138]:
ib.show(pc)

{'a': 4, 'b': 1}
{'a': 1, 'b': 6}


In [2]:
#in pipline: pcoll | Collapse(pipeline, k_fields, schema_str)

# class MaxFinder(beam.DoFn):
#     def process(self, element, pipeline, k_field, schema_str):
#         schema_fields = schema_str.split(',')
#         field_max = (
#             records 
#             | field >> beam.GroupBy(lambda x: x[k_field])
#             .aggregate_field(lambda x: x[field], max, field)
#             )

        


#         yield 

class Collapse(beam.PTransform):
    def __init__(self, pipeline, k_fields, schema_str):
        self.pipeline = pipeline
        self.k_fields = k_fields
        self.schema_str = schema_str

    def expand(self, records):
        
        schema_fields = self.schema_str.split(',')
        fields = [s.split(':')[0] for s in schema_fields if s.split(':')[0] not in self.k_fields]
        
        # result = {}

        for k_field in self.k_fields:
            for field in fields:
                # field_max = records | beam.Map(MaxFinder)
                field_max = (
                    records 
                    | field >> beam.GroupBy(lambda x: x[k_field])
                    .aggregate_field(lambda x: x[field], max, field)
                    )
            
                yield field_max

        #         result.update({field: field_max})
        #         # print(result)
        #     # (result) | beam.CoGroupByKey()
        #     # ParDo(MaxFinder(), pipeline, k_field, schema_str)
        # if result:
        #     yield result


In [3]:
k_fields = ['a']
ss = 'a:INTEGER,b:INTEGER,c:INTEGER,d:INTEGER'
schema_fields = ss.split(',')
fields = [s.split(':')[0] for s in schema_fields if s.split(':')[0] not in k_fields]
fields

['b', 'c', 'd']

In [4]:
p = Pipeline(InteractiveRunner())
pcoll = p | beam.Create([{'a':4,'b':1,'c':3,'d':6}, {'a':4,'b':7,'c':4,'d':3}, {'a':1,'b':6,'c':8,'d':2}])

result = pcoll | Collapse(pipeline = p, k_fields = ['a'], schema_str = 'a:INTEGER,b:INTEGER,c:INTEGER,d:INTEGER') | beam.Map(print)


usage: ipykernel_launcher [-h] [--dataflow_endpoint DATAFLOW_ENDPOINT]
                          [--project PROJECT] [--job_name JOB_NAME]
                          [--staging_location STAGING_LOCATION]
                          [--temp_location TEMP_LOCATION] [--region REGION]
                          [--service_account_email SERVICE_ACCOUNT_EMAIL]
                          [--no_auth] [--template_location TEMPLATE_LOCATION]
                          [--label LABELS] [--update]
                          [--transform_name_mapping TRANSFORM_NAME_MAPPING]
                          [--enable_streaming_engine]
                          [--dataflow_kms_key DATAFLOW_KMS_KEY]
                          [--create_from_snapshot CREATE_FROM_SNAPSHOT]
                          [--flexrs_goal {COST_OPTIMIZED,SPEED_OPTIMIZED}]
                          [--dataflow_service_option DATAFLOW_SERVICE_OPTIONS]
                          [--enable_hot_key_logging]
ipykernel_launcher: error: argument --flex

Traceback (most recent call last):
  File "C:\Program Files\Python38\lib\argparse.py", line 1800, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "C:\Program Files\Python38\lib\argparse.py", line 2006, in _parse_known_args
    start_index = consume_optional(start_index)
  File "C:\Program Files\Python38\lib\argparse.py", line 1946, in consume_optional
    take_action(action, args, option_string)
  File "C:\Program Files\Python38\lib\argparse.py", line 1858, in take_action
    argument_values = self._get_values(action, argument_strings)
  File "C:\Program Files\Python38\lib\argparse.py", line 2390, in _get_values
    self._check_value(action, value)
  File "C:\Program Files\Python38\lib\argparse.py", line 2446, in _check_value
    raise ArgumentError(action, msg % args)
argparse.ArgumentError: argument --flexrs_goal: invalid choice: 'C:\\Users\\markc\\AppData\\Local\\Temp\\tmp-4720F2lZSuoMK3iF.json' (choose from 'COST_OPTIMIZED', 'SPEED_OPTIMIZED

TypeError: object of type 'NoneType' has no len()

In [170]:
p.run()

In [156]:
ib.show(result)

AttributeError: 'dict' object has no attribute 'element_type'

In [207]:
p = Pipeline(InteractiveRunner())
pcoll = p | beam.Create([{'a':4,'b':1,'c':3,'d':6}, {'a':4,'b':7,'c':4,'d':3}, {'a':1,'b':6,'c':8,'d':2}])

res = []

for k in ['a']:
    for f in ['c','d']:
        gf = (
            pcoll 
            | f >> beam.GroupBy(lambda x: x[k]).aggregate_field(lambda x: x[f], max, f)
            | k+f >> beam.Map(print)
            # | k+k+f+f >> beam.CoGroupByKey()
            # | beam.Map(print)
            )
        res.append(gf)

res | beam.Map(print)

# ({'c':gc,'d':gd}) | beam.CoGroupByKey() | beam.Map(print)

# (4, {'c': [4], 'd': [6]})
# (1, {'c': [8], 'd': [2]})
p.run()

usage: ipykernel_launcher [-h] [--dataflow_endpoint DATAFLOW_ENDPOINT]
                          [--project PROJECT] [--job_name JOB_NAME]
                          [--staging_location STAGING_LOCATION]
                          [--temp_location TEMP_LOCATION] [--region REGION]
                          [--service_account_email SERVICE_ACCOUNT_EMAIL]
                          [--no_auth] [--template_location TEMPLATE_LOCATION]
                          [--label LABELS] [--update]
                          [--transform_name_mapping TRANSFORM_NAME_MAPPING]
                          [--enable_streaming_engine]
                          [--dataflow_kms_key DATAFLOW_KMS_KEY]
                          [--create_from_snapshot CREATE_FROM_SNAPSHOT]
                          [--flexrs_goal {COST_OPTIMIZED,SPEED_OPTIMIZED}]
                          [--dataflow_service_option DATAFLOW_SERVICE_OPTIONS]
                          [--enable_hot_key_logging]
ipykernel_launcher: error: argument --flex

Traceback (most recent call last):
  File "C:\Program Files\Python38\lib\argparse.py", line 1800, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "C:\Program Files\Python38\lib\argparse.py", line 2006, in _parse_known_args
    start_index = consume_optional(start_index)
  File "C:\Program Files\Python38\lib\argparse.py", line 1946, in consume_optional
    take_action(action, args, option_string)
  File "C:\Program Files\Python38\lib\argparse.py", line 1858, in take_action
    argument_values = self._get_values(action, argument_strings)
  File "C:\Program Files\Python38\lib\argparse.py", line 2390, in _get_values
    self._check_value(action, value)
  File "C:\Program Files\Python38\lib\argparse.py", line 2446, in _check_value
    raise ArgumentError(action, msg % args)
argparse.ArgumentError: argument --flexrs_goal: invalid choice: 'C:\\Users\\markc\\AppData\\Local\\Temp\\tmp-4720nTrq6x89vXoU.json' (choose from 'COST_OPTIMIZED', 'SPEED_OPTIMIZED

TypeError: object of type 'NoneType' has no len()

In [29]:
p = Pipeline(InteractiveRunner())
pcoll = p | beam.Create([{'a':4,'b':1,'c':3,'d':6}, {'a':4,'b':7,'c':4,'d':3}, {'a':1,'b':6,'c':8,'d':2}])
pcoll | beam.GroupBy(lambda x: x['a']) | beam.FlatMap(lambda elements: elements) | beam.Map(print)

<PCollection[[29]: Map(print).None] at 0x1ab25265730>

In [30]:
p.run()
# ib.show(pcoll)

4
[{'a': 4, 'b': 1, 'c': 3, 'd': 6}, {'a': 4, 'b': 7, 'c': 4, 'd': 3}]
1
[{'a': 1, 'b': 6, 'c': 8, 'd': 2}]


In [137]:
p = Pipeline(InteractiveRunner())
pcoll = p | beam.Create([{'a':4,'b':1,'c':3,'d':6}, {'a':4,'b':7,'c':4,'d':3}, {'a':1,'b':6,'c':8,'d':2}])

gc = (
    pcoll 
    | beam.GroupBy(lambda x: x['a']).aggregate_field(lambda x: x['c'], max, 'c')
    )

gd = (
    pcoll
    | 'l' >> beam.GroupBy(lambda x: x['a']).aggregate_field(lambda x: x['d'], max, 'd')
    )

({'c':gc,'d':gd}) | beam.CoGroupByKey() | beam.Map(print)

# (4, {'c': [4], 'd': [6]})
# (1, {'c': [8], 'd': [2]})
p.run()

(4, {'c': [4], 'd': [6]})
(1, {'c': [8], 'd': [2]})


In [48]:
class PivotRow(beam.DoFn):
    def process(self, element, k_fields, p_fields, v_fields):
        
        # start with the key_fields, unchanged
        pivoted_record = [(k,v) for k,v in element.items() if k in k_fields]
        
        # then update with new pivoted fields 
        for p_field in p_fields:
            for v_field in v_fields:
                col_name = element[p_field] + '_' + v_field
                col_name = col_name.replace(' ', '_')
                value = element[v_field]
                pivoted_record.append((col_name,value))

        yield tuple(pivoted_record)

# input:
# elems = [
#     {'id': '27047594', 'name': 'Beautiful 5100 SQ FT Industrial Brooklyn Loft', 'host_id': '319077', 'host_name': 'Shell', 'neighbourhood_group': 'Brooklyn', 'neighbourhood': 'Clinton Hill', 'latitude': '40.6855', 'longitude': '-73.96112', 'room_type': 'Entire home/apt', 'price': '999', 'minimum_nights': '1', 'number_of_reviews': '2', 'last_review': '11/18/2018', 'reviews_per_month': '0.22', 'calculated_host_listings_count': '4', 'availability_365': '365'},
#     {'id': '13019851', 'name': 'Beautiful Brooklyn Bedroom!', 'host_id': '59093909', 'host_name': 'Katherine E', 'neighbourhood_group': 'Brooklyn', 'neighbourhood': 'Clinton Hill', 'latitude': '40.69525', 'longitude': '-73.96604', 'room_type': 'Private room', 'price': '45', 'minimum_nights': '1', 'number_of_reviews': '3', 'last_review': '7/14/2016', 'reviews_per_month': '0.08', 'calculated_host_listings_count': '1', 'availability_365': '0'}, 
#     {'id': '26989977', 'name': 'Beautiful Clinton Hill 1-BR in luxury building', 'host_id': '24822059', 'host_name': 'Jessica', 'neighbourhood_group': 'Mars', 'neighbourhood': 'Clinton Hill', 'latitude': '40.68214', 'longitude': '-73.96536', 'room_type': 'Entire home/apt', 'price': '119', 'minimum_nights': '6', 'number_of_reviews': '2', 'last_review': '8/29/2018', 'reviews_per_month': '0.18', 'calculated_host_listings_count': '1', 'availability_365': '0'}]
# k_fields = ['neighbourhood_group']
# p_fields = ['neighbourhood']
# v_fields = ['price']

# output:
# [('neighbourhood_group', 'Brooklyn'), ('Clinton_Hill_price', '999')]
# [('neighbourhood_group', 'Brooklyn'), ('Clinton_Hill_price', '45')]
# [('neighbourhood_group', 'Mars'), ('Clinton_Hill_price', '119')]


In [68]:
class PivotRow(beam.DoFn):
    def process(self, element, k_fields, p_fields, v_fields):
        
        # start with the key_fields, unchanged
        pivoted_record = {k:v for k,v in element.items() if k in k_fields}
        
        # then update with new pivoted fields 
        for p_field in p_fields:
            for v_field in v_fields:
                col_name = element[p_field] + '_' + v_field
                col_name = col_name.replace(' ', '_')
                value = element[v_field]
                pivoted_record.update({col_name:value})

        yield pivoted_record

In [69]:
elems = [
    {'id': '27047594', 'name': 'Beautiful 5100 SQ FT Industrial Brooklyn Loft', 'host_id': '319077', 'host_name': 'Shell', 'neighbourhood_group': 'Brooklyn', 'neighbourhood': 'Clinton Hill', 'latitude': '40.6855', 'longitude': '-73.96112', 'room_type': 'Entire home/apt', 'price': '999', 'minimum_nights': '1', 'number_of_reviews': '2', 'last_review': '11/18/2018', 'reviews_per_month': '0.22', 'calculated_host_listings_count': '4', 'availability_365': '365'},
    {'id': '13019851', 'name': 'Beautiful Brooklyn Bedroom!', 'host_id': '59093909', 'host_name': 'Katherine E', 'neighbourhood_group': 'Brooklyn', 'neighbourhood': 'Clinton Hill', 'latitude': '40.69525', 'longitude': '-73.96604', 'room_type': 'Private room', 'price': '45', 'minimum_nights': '1', 'number_of_reviews': '3', 'last_review': '7/14/2016', 'reviews_per_month': '0.08', 'calculated_host_listings_count': '1', 'availability_365': '0'}, 
    {'id': '26989977', 'name': 'Beautiful Clinton Hill 1-BR in luxury building', 'host_id': '24822059', 'host_name': 'Jessica', 'neighbourhood_group': 'Mars', 'neighbourhood': 'Clinton Hill', 'latitude': '40.68214', 'longitude': '-73.96536', 'room_type': 'Entire home/apt', 'price': '119', 'minimum_nights': '6', 'number_of_reviews': '2', 'last_review': '8/29/2018', 'reviews_per_month': '0.18', 'calculated_host_listings_count': '1', 'availability_365': '0'}]
k_fields = ['neighbourhood_group']
p_fields = ['neighbourhood']
v_fields = ['price']

In [206]:
import argparse
parser = argparse.ArgumentParser()
args, unknown = parser.parse_known_args()
parser.add_argument('--flexrs_goal, SPEED_OPTIMIZED')

_StoreAction(option_strings=['--flexrs_goal, SPEED_OPTIMIZED'], dest='flexrs_goal, SPEED_OPTIMIZED', nargs=None, const=None, default=None, type=None, choices=None, help=None, metavar=None)

In [200]:
import re
pp = Pipeline(InteractiveRunner())
pc = (
    pp 
    | beam.Create(elems) 
    | beam.ParDo(PivotRow(), k_fields, p_fields, v_fields) 
    # | beam.Map(lambda x: (x['neighbourhood_group'], {}))
    | beam.GroupBy(lambda x: x['neighbourhood_group'])
    # | beam.CombineValues(lambda x: max(x['Clinton_Hill_price']))
    # | beam.Filter(lambda tup: tup[0]=='Mars')#lambda x: x['neighbourhood_group'])
    # | beam.transforms.util.Values()
    # | beam.Map()
    # | beam.Map(lambda elems: set([tuple([k,v]) for elem in elems for k,v in elem.items()]))
    # | beam.CoGroupByKey(pipeline=pp)
    # | beam.CombinePerKey(max)
    # | beam.CombinePerKey(lambda g: max(g) if len(re.findall('price', g))>0 else None)#.with_input_types(List[Tuple[Any, Any]]) 
    # | beam.Map(print)
    )

# with orig PivotRow List[dict] output, ...
# ('Brooklyn', [{'neighbourhood_group': 'Brooklyn', 'Clinton_Hill_price': '999'}, {'neighbourhood_group': 'Brooklyn', 'Clinton_Hill_price': '45'}])
# ('Mars', [{'neighbourhood_group': 'Mars', 'Clinton_Hill_price': '119'}])

In [201]:
pp.run()

KeyError: "neighbourhood_group [while running '[200]: GroupBy(key)/Map(<lambda at core.py:2734>)']"

In [130]:
ib.show(pc)

In [ ]:
def unify(set_of_tups):
    


In [ ]:
# import re
# pp = Pipeline(InteractiveRunner())
# pc = (
#     pp 
#     | beam.Create(elems) 
#     | beam.ParDo(PivotRow(), k_fields, p_fields, v_fields) 
#     | beam.GroupBy(lambda x: x['neighbourhood_group'])
#     | beam.transforms.util.Values()
#     | beam.Map(lambda elems: [tuple([k,v]) for elem in elems for k,v in elem.items()])
#     | beam.Map(print)
#     )

# # with orig PivotRow List[dict] output, ...
# # pp.run()

# # [('neighbourhood_group', 'Brooklyn'), ('Clinton_Hill_price', '999'), ('neighbourhood_group', 'Brooklyn'), ('Clinton_Hill_price', '45')]
# # [('neighbourhood_group', 'Mars'), ('Clinton_Hill_price', '119')]

In [117]:
qq = Pipeline(InteractiveRunner())
pc = (
    qq 
    | beam.Create(elems) 
    | beam.ParDo(PivotRow(), k_fields, p_fields, v_fields) 
    # for key in key_fields ... for f not in key_fields
    | beam.GroupBy(lambda x: x['neighbourhood_group'])
    | beam.transforms.util.Values()
    | beam.Map(lambda elems: [tuple([k,v]) for elem in elems for k,v in elem.items()])
    | beam.GroupBy(lambda x: x[1] if x[0] == 'neighbourhood_group' else None)
    .aggregate_field(lambda x: x[1] if x[0] == 'Clinton_Hill_price' else None, lambda x: , 'c')
    | beam.Map(print)
    )

In [118]:
qq.run()

TypeError: '>' not supported between instances of 'NoneType' and 'NoneType'

In [ ]:
import re
pp = Pipeline(InteractiveRunner())
pc = (
    pp 
    | beam.Create(elems) 
    | beam.ParDo(PivotRow(), k_fields, p_fields, v_fields) 
    | beam.GroupBy(lambda x: x['neighbourhood_group'])
    | beam.transforms.util.Values()
    | beam.Map(lambda elems: [tuple([k,v]) for elem in elems for k,v in elem.items()])
    | beam.Map(print)
    )

# with orig PivotRow List[dict] output, ...
# pp.run()

# [('neighbourhood_group', 'Brooklyn'), ('Clinton_Hill_price', '999'), ('neighbourhood_group', 'Brooklyn'), ('Clinton_Hill_price', '45')]
# [('neighbourhood_group', 'Mars'), ('Clinton_Hill_price', '119')]

In [78]:
input = [{'neighbourhood_group': 'Brooklyn', 'Clinton_Hill_price': '999'}, {'neighbourhood_group': 'Brooklyn', 'Clinton_Hill_price': '45'}]

In [ ]:
def max_per_key(list_of_dicts, k_fields):
    new_record = {}
    for dict in list_of_dicts:
        


In [77]:
pp.run()

TypeError: Expected a CombineFn or callable, got <apache_beam.transforms.core._ReiterableChain object at 0x000001F4A95301C0>

In [29]:
pp.run() 

#output of new PivotRow(), with tuples instead of dicts

# [('neighbourhood_group', 'Brooklyn'), ('Clinton_Hill_price', '999')]
# [('neighbourhood_group', 'Brooklyn'), ('Clinton_Hill_price', '45')]
# [('neighbourhood_group', 'Mars'), ('Clinton_Hill_price', '119')]

# next, I want to eliminate that middle row, because only max price per neighbourhood for each neighbourhood_group (which is the key_field)

[('neighbourhood_group', 'Brooklyn'), ('Clinton_Hill_price', '999')]
[('neighbourhood_group', 'Brooklyn'), ('Clinton_Hill_price', '45')]
[('neighbourhood_group', 'Mars'), ('Clinton_Hill_price', '119')]


In [ ]:
{'a':4,'b':1,'c':3,'d':6}

In [ ]:
q = Pipeline(InteractiveRunner())
pcoll = q | beam.Create([{'a':4,'b':1,'c':3,'d':6}, {'a':4,'b':7,'c':4,'d':3}, {'a':1,'b':6,'c':8,'d':2}])

gc = (
    pcoll 
    | beam.GroupBy(lambda x: x['a']).aggregate_field(lambda x: x['c'], max, 'c')
    # | beam.Map(print)
    )

gd = (
    pcoll
    | 'l' >> beam.GroupBy(lambda x: x['a']).aggregate_field(lambda x: x['d'], max, 'd')
    # | 'l2' >> beam.Map(print)
    )

({'c':gc,'d':gd}) | beam.CoGroupByKey() | beam.Map(print)

# (4, {'c': [4], 'd': [6]})
# (1, {'c': [8], 'd': [2]})

___
___

In [66]:
pipe = Pipeline(InteractiveRunner())
pcoll = pipe | beam.Create([{'a':4,'b':1,'c':3,'d':6}, {'a':4,'b':7,'c':4,'d':3}, {'a':1,'b':6,'c':8,'d':2}])

grouped = (
    pcoll 
    | beam.GroupBy(lambda x: x['a'])
    )
grouped | beam.Map(lambda keyed_tup: keyed_tup[1]) | beam.CombinePerKey(max) | beam.Map(print)


TypeCheckError: Input type hint violation at GroupByKey: expected Tuple[TypeVariable[K], TypeVariable[V]], got Iterable[Dict[str, int]]
Full type hint:
IOTypeHints[inputs=((Tuple[TypeVariable[K], TypeVariable[V]],), {}), outputs=((Tuple[TypeVariable[K], Iterable[TypeVariable[V]]],), {})]
File "<frozen importlib._bootstrap>", line 671, in _load_unlocked
File "<frozen importlib._bootstrap_external>", line 848, in exec_module
File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
File "C:\Users\markc\.virtualenvs\onboard_beam-R5rNJSK6\lib\site-packages\apache_beam\transforms\core.py", line 2542, in <module>
    class GroupByKey(PTransform):
File "C:\Users\markc\.virtualenvs\onboard_beam-R5rNJSK6\lib\site-packages\apache_beam\typehints\decorators.py", line 775, in annotate_input_types
    th = getattr(f, '_type_hints', IOTypeHints.empty()).with_input_types(

based on:
  IOTypeHints[inputs=None, outputs=((Tuple[TypeVariable[K], Iterable[TypeVariable[V]]],), {})]
  File

In [6]:
pipe.run()

[{'a': 4, 'b': 1, 'c': 3, 'd': 6}, {'a': 4, 'b': 7, 'c': 4, 'd': 3}]
[{'a': 1, 'b': 6, 'c': 8, 'd': 2}]


In [8]:
from collections import namedtuple

In [53]:
# nt = namedtuple(field_names = ['a','b','c','d'])
elem = {'a': 4, 'b': 7, 'c': 4, 'd': 3}
tups = []
for k,v in elem.items():
    tups.append(tuple([k,v]))

In [61]:
elems = [{'a': 4, 'b': 7, 'c': 4, 'd': 3}]
# lambda elem: (elem.values() if elem in k_fields)
# output --> (keys, dict)

In [65]:
tuple(elem.values() for elem in elems.values() if k in k_fields)

AttributeError: 'list' object has no attribute 'values'

In [55]:
[tuple([k,v]) for k,v in elem.items()]

[('a', 4), ('b', 7), ('c', 4), ('d', 3)]

In [56]:
k_fields = ['a']
[tuple([k,v]) for k,v in elem.items() if k not in k_fields]

[('b', 7), ('c', 4), ('d', 3)]

In [15]:
tups

[('a', 4), ('b', 7), ('c', 4), ('d', 3)]

In [4]:
print(ib.show(grouped))

None


In [ ]:
class MaxPerCol(beam.PTransform):
    """ if an element is a python dictionary, this should
    get the max for each col """
    def expand(self, pcollection):

        for k_field in k_fields:
            pcollection | beam.GroupBy(lambda row: row[k_field])


        for key in pcollection.keys()
            pcollection = pcollection | beam.GroupBy()


In [2]:
p = Pipeline(InteractiveRunner())

In [3]:
table_spec = bigquery.TableReference(
    projectId='practice-springml',
    datasetId='baby_names',
    tableId='pivoted_table')

In [ ]:
def get_max_items(dicts):
    for dict in dicts:
        for k,v in dict.items():
            

  return set.intersection(*(sets or [set()]))

with beam.Pipeline() as pipeline:
  common_items = (
      pipeline
      | 'Create produce' >> beam.Create([
          {'🍓', '🥕', '🍌', '🍅', '🌶️'},
          {'🍇', '🥕', '🥝', '🍅', '🥔'},
          {'🍉', '🥕', '🍆', '🍅', '🍍'},
          {'🥑', '🥕', '🌽', '🍅', '🥥'},
      ])
      | 'Get common items' >> beam.CombineGlobally(get_common_items)
      | beam.Map(print))

In [64]:
z = Pipeline(InteractiveRunner())
pcoll = (
    z 
    | beam.Create([{'a':4,'b':1,'c':3,'d':6}, {'a':4,'b':7,'c':4,'d':3}, {'a':1,'b':6,'c':8,'d':2}])
    | beam.GroupBy(lambda x: x['a']).aggregate_field(lambda x: x.keys(), max, lambda x: x.keys())
    | beam.Map(print)
    )

In [65]:
z.run()

TypeError: cannot pickle 'dict_keys' object [while running '[64]: _GroupAndAggregate/Map(<lambda at core.py:2790>)']

In [60]:
beam.CombinePerKey(sum)

<CombinePerKey(PTransform) label=[CombinePerKey(sum)] at 0x21891851b80>

In [51]:
q = Pipeline(InteractiveRunner())
pcoll = (
    q 
    | beam.Create([{'a':4,'b':1,'c':3,'d':6}, {'a':4,'b':7,'c':4,'d':3}, {'a':1,'b':6,'c':8,'d':2}])
    | beam.GroupBy(lambda x: x['a'])
    .aggregate_field(lambda x: x['c'], max, 'c')
    .aggregate_field(lambda x: x['d'], max, 'd')
    # | beam.combiners.Top().PerKey(n=1)
    | beam.Map(print)
    )

In [52]:
q.run()

Result(key=4, c=4, d=6)
Result(key=1, c=8, d=2)


In [4]:
q = Pipeline(InteractiveRunner())
pcoll = q | beam.Create([{'a':4,'b':1,'c':3,'d':6}, {'a':4,'b':7,'c':4,'d':3}, {'a':1,'b':6,'c':8,'d':2}])

gc = (
    pcoll 
    | beam.GroupBy(lambda x: x['a']).aggregate_field(lambda x: x['c'], max, 'c')
    # | beam.Map(print)
    )

gd = (
    pcoll
    | 'l' >> beam.GroupBy(lambda x: x['a']).aggregate_field(lambda x: x['d'], max, 'd')
    # | 'l2' >> beam.Map(print)
    )

({'c':gc,'d':gd}) | beam.CoGroupByKey() | beam.Map(print)

# (4, {'c': [4], 'd': [6]})
# (1, {'c': [8], 'd': [2]})

<PCollection[[4]: Map(print).None] at 0x221073a1a30>

In [5]:
q.run()

(4, {'c': [4], 'd': [6]})
(1, {'c': [8], 'd': [2]})


In [21]:
ib.show((s1,s2))

Result(key=4, c=4)
Result(key=1, c=8)
Result(key=4, d=6)
Result(key=1, d=2)


In [8]:
q.run()
# ib.show(pcoll)

Result(key=4, c=4)
Result(key=1, c=8)
Result(key=4, d=6)
Result(key=1, d=2)


In [6]:
pcollection = [{'a':4,'b':1,'c':3,'d':6}, {'a':4,'b':7,'c':4,'d':3}, {'a':1,'b':6,'c':8,'d':2}]

In [9]:
for element in pcollection:
    print(element.keys())
    element.keys()

dict_keys(['a', 'b', 'c', 'd'])
dict_keys(['a', 'b', 'c', 'd'])
dict_keys(['a', 'b', 'c', 'd'])


In [ ]:
class MaxPerCol(beam.PTransform):
    """ if an element is a python dictionary, this should
    get the max for each col """
    def expand(self, pcollection):

        for k_field in k_fields:
            pcollection | beam.GroupBy(lambda row: row[k_field])


        for key in pcollection.keys()
            pcollection = pcollection | beam.GroupBy()


In [4]:
pc = (
    p 
    | ReadFromBigQuery(table=table_spec, project='practice-springml', method="DIRECT_READ")
    # | beam.Map(lambda x: x['name'] == 'David')
    # | beam.combiners.Top.Largest(2)
    # | beam.GroupBy(lambda x: x['name']).aggregate_field('F_number', sum, 'F')
    | beam.GroupBy(lambda x: x['name'])
    .aggregate_field(lambda x: x['F_number'], sum, 'total_F')
    .aggregate_field(lambda x: x['M_number'], sum, 'total_M')
    | beam.Map(print)
    )

# Result(key='F', total=152173203)
# Result(key='M', total=164705298)

C:\Users\markc\.virtualenvs\onboard_beam-R5rNJSK6\lib\site-packages\apache_beam\io\gcp\bigquery.py:2439: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  project_id = pcoll.pipeline.options.view_as(GoogleCloudOptions).project


In [5]:
p.run()

KeyError: "F_number [while running '[4]: _GroupAndAggregate/Map(<lambda at core.py:2790>)']"

In [1]:
grouped = pc | beam.GroupBy('name')

NameError: name 'pc' is not defined

In [21]:
# ib.show(pc, n=2)
# p.run()

In [18]:
(1 << 20)

1048576

In [17]:
64 * (1 << 20)

67108864

In [19]:
2**20

1048576